<h1><center>Project</center></h1>
<h2><center>Micro-452 : Basics of Mobile Robotics</center></h2>
<h3><center>Group 11</center></h3>
<h3><center>Benoist Marguerite, Chang Chun-Tzu, Maradan Théodore, Tambourin Noé</center></h3>

## 1. Introduction

This report presents the project done in the Basics of Mobile Robotics class. The project consists in programming a robot (the "Thymio") to navigate in an environment containing a set of (expected and unexpected) obstacles.

To do so, the robot should first be able to build a global map of its environment using a camera (<b>vision</b>). From the map, the robot should determine the shortest path from its starting point to its goal (<b>global naviagation</b>). Then, <b>filtering</b> the information coming from the camera and from the built-in sensors the robot should travel to a goal point set anywhere on the map using <b>motor control</b>. Finally, the robot should avoid any unexpected obstacle on its path using <b>local navigation</b>.

The global structure of the project with the main fluxes of information between the different parts of it can be seen in Figure 1.

<img src="Report_Figures/Global_Diagram.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 1: Global Diagram of the Project</b></p>

The sections below describe in detail our solution to the problem and the choices and assumptions made through the realization of this project.

In particular, section 2 describes the implementation of the diagram above using a state machine as well as the environment chosen.

## 2. Choices & Assumptions

In order to combine the different parts of the project, we decided to use a state machine using two states: Obstacle avoidance & global navigation. The former is used to get around unforeseen obstacles (obstacles that were not included in the global map) while the latter is used to follow the shortest path from the starting point to the goal point. An illustration of the state machine used is provided in Figure 2.

<font color='red'>INCLUDE FIGURE STATE MACHINE</font>

<img src="Report_Figures/State_Machine_Diagram.jpeg" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 2: State Machine used for the Project</b></p>

The environment constituting the global map was chosen to be blank with 2D black obstacles (obstacles without thickness). The contrast between the map and the obstacles allows for a better detection of the obstacles by the camera. In contrast, the unexpected obstacles are 3D (non-null thickness).

This distinction ensures that the obstacles detected by the proximity sensors are unexpected obstacles and avoids confusion with obstacles being part of the global map.

<font color='red'>INCLUDE FIGURE ENVIRONMENT</font>

<img src="Report_Figures/Environment.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 3: Environment</b></p>

In order to identify the robot and the goal point, ArUco markers are used. These synthetic markers are easy to identify using an appropriate library (OpenCV). Moreover, they ensure a reliable robot pose acquisition by the camera.

<font color='red'>INCLUDE FIGURE ARUCO MARKER</font>

<img src="Report_Figures/ArUco_Marker.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 4: ArUco Markers</b></p>

Finally, the camera is fixed above the global map so that it sees the entire environment while avoiding as much as possible distortions if the map due to the camera not being parallel with the plane of the map.

<font color='red'>INCLUDE FIGURE ENVIRONMENT + CAMERA</font>

<img src="Report_Figures/Environment_Camera.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 5: Environment with Camera</b></p>

<font color='red'>INCLUDE SENTENCE CODE IS ONLY PART OF IT</font>

## 3. Vision

### 3.1 Global Map

In [ ]:
def compute_globalmap(self, image, start_id=0, goal_id=1):
        # Generate globalmap : 0:free 1:occupied
        height, width = image.shape[:2]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Otsu's thresholding after Gaussian filtering
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Generate Global map
        # from 0,255 convert to 0(free),1(occupied);change to (x,y) coordinate
        temp_globalmap = np.transpose(np.array(thresh < 255, dtype=int))

        # Detect start and goal by aruco marker
        # Return start and goal position in (x,y) coordinate; if not detected return None
        # start_id: start aruco marker id
        # goal_id=: goal aruco marker id

        # Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        # Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
        start = None
        goal = None
        s_start = 50 #start margin
        s_goal = 20 #goal margin
        if ids is not None:
            for corner, number in zip(corners, ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                centerpoint = ((topLeft[0] + bottomRight[0]) / 2, (topLeft[1] + bottomRight[1]) / 2)
                side_length=math.dist(topLeft, topRight)/2
                self.side_length=side_length
                if number == start_id:
                    start = centerpoint
                    #get rid of aruco marker of start point
                    print("start:",start)
                    x_small=int(centerpoint[0]-side_length)-s_start
                    x_large=int(centerpoint[0]+side_length)+s_start
                    y_small =int(centerpoint[1]-side_length)-s_start
                    y_large=int(centerpoint[1]+side_length)+s_start
                    if x_small<0: x_small=0
                    if y_small<0:y_small=0
                    if x_large>=width:x_large=width-1
                    if y_large>=height:y_large=height-1
                    temp_globalmap[x_small:x_large,y_small:y_large] = 0

                elif number == goal_id:
                    goal = centerpoint
                    #get rid of aruco marker of goal point
                    print("goal:",goal)
                    x_small = int(centerpoint[0] - side_length) - s_goal
                    x_large = int(centerpoint[0] + side_length) + s_goal
                    y_small = int(centerpoint[1] - side_length) - s_goal
                    y_large = int(centerpoint[1] + side_length) + s_goal
                    if x_small < 0: x_small = 0
                    if y_small < 0: y_small = 0
                    if x_large >= width: x_large = width - 1
                    if y_large >= height: y_large = height - 1
                    temp_globalmap[x_small:x_large, y_small:y_large] = 0
                    
                else:
                    pass
        else:
            pass

        globalmap = temp_globalmap.copy()
        if start is None:
            print("NOT detect start point")
        else:
            self.start = start
        if goal is None:
            print("NOT detect goal point")
        else:
            self.goal = goal

        return globalmap
        

The first step of the project is to construct the map with the vision. We fisrt use Otsu's threshold algorithm to recognized the black global obstacles and then use aruco marker to detect goal point and start point which is where robot is at the beginning.

#### 3.1.1 Otsu's threshold method

Image thresholding binarizes a grayscale image based on pixel values and a threshold value. For example, when the pixel value is samller than the threshold value, this pixel will be marked as black. When the pixel value is larger then the threshold value, this pixel will be marked as white. The threshold algorithm we choosed is Otsu's threshold. Comparing to simple threshold method which use an manually choosen value as threshold, Otsu's threshold method [1] tries to find a global optimal threshold value from image histogram which can minimize the weighted within-class variance given by:

$\sigma_w^2(t) = q_1(t)\sigma_1^2(t)+q_2(t)\sigma_2^2(t)\$

where

$q_1(t) = \sum_{i=1}^{t} P(i) \quad \& \quad q_2(t) = \sum_{i=t+1}^{I} P(i)$

$\mu_1(t) = \sum_{i=1}^{t} \frac{iP(i)}{q_1(t)} \quad \& \quad \mu_2(t) = \sum_{i=t+1}^{I} \frac{iP(i)}{q_2(t)}$

$\sigma_1^2(t) = \sum_{i=1}^{t} [i-\mu_1(t)]^2 \frac{P(i)}{q_1(t)} \quad \& \quad \sigma_2^2(t) = \sum_{i=t+1}^{I} [i-\mu_2(t)]^2 \frac{P(i)}{q_2(t)}$

In our map, we have black (global obstacle) and white (free move space) pixels, so the image histogram will have two peaks. Then, Otsu's threshold method find a value lying between two peaks to minimizes both classes variances which is the optimal threshold value we used. By using Otsu's threshold, it will help us to choose the optimal threshold value rather than manually choose the threshold value evrytime depending on the environment when using simple threshold method. Following figures show the difference between using two methods with different environment,
<table><tr>
<td> <img src="Report_Figures/map1.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 1</b></p></td>
<td><img src="Report_Figures/Otsus_thresholding_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Otsu's threshold</b></p></td>
<td><img src="Report_Figures/simple_threshold_method_50_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Simple threshold, threshold=50</b></p></td>
<td><img src="Report_Figures/simple_threshold_method_127_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Simple threshold, threshold=127</b></p></td>
</tr></table>

<table><tr>
<td> <img src="Report_Figures/map2.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 2</b></p></td>
<td><img src="Report_Figures/Otsus_thresholding_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Otsu's threshold</b></p></td>
<td><img src="Report_Figures/simple_threshold_method_50_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Simple threshold, threshold=50</b></p></td>
<td><img src="Report_Figures/simple_threshold_method_127_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Simple threshold, threshold=127</b></p></td>
</tr></table>
<p style='text-align: center;'><b>Figure 6: Effect of different thresholds</b></p>


We can observe that Otsu's threshold shows stable performance with differnt evironment. Simple threshold method performs better with threshold value 127 in Map 1, however, in Map 2, threshold value 50 perfomrs better. Therefore, using simple threshold method may need to manually adjust the threshold value when the environment changed.

#### 3.1.2 ArUco Marker

Then, we need to detect goal point and start point which is the robot position for constructing global map. To detect the position of objects, we use aruco marker which is fast and robust.

Aruco marker [2] provides the position of its four conners and uses its inner binary matrix to determine its id. To detect an aruco marker, it fisrt apllies adaptive threshold and then extractes the contours of shapes in the image. If the extracted shape is not a convex or not simillar to a square shape, it will be discarded. After selecting the candidates of marker, it need to extracte the inner bits of each candidates. Otsu's thresholding  is used to first seperate black and white pixels in the image. Then, the extracted shape is divided to differnt cell according to the size of marker and boarder. The cell which contains more black pixels will be considered as a black bit, vice versa. Finally, we can get four coners and id by binary codification.

Below shows some example of ArUco marker detection.

<table><tr>
<td> <img src="Report_Figures/arucomarker_map1.png" style='text-align: left' width="320" height="416;"/><p style='text-align: center;'><b>Map 1</b></p></td>
<td> <img src="Report_Figures/arucomarker_map2.png" style='text-align: left' width="320" height="416;"/><p style='text-align: center;'><b>Map 2</b></p></td>
</tr></table>
<p style='text-align: center;'><b>Figure 7: Detection of ArUco Markers</b></p>

#### 3.1.3 Add margin

In [ ]:
def add_margin(self,globalmap,robot_size=70):
       #object inflation, add margin to global map
       #robot_size: robot_size in mm
       #return global map with margin

        # Compute pixel to distance value
        pixel_to_distance = self.compute_pixel_to_distance()

        # obstacle inflation
        # 3x3 structuring element with connectivity 2
        struct = ndimage.generate_binary_structure(2, 2)
        # robot_size:radius of robot(unit:mm)
        margin = int(robot_size / pixel_to_distance) + 1  # margin at least need to be 1, can't be 0
        globalmap = ndimage.binary_dilation(globalmap, structure=struct, iterations=margin).astype(globalmap.dtype)

        self.globalmap = globalmap

        return self.globalmap

Since the robot is not only a point, we need to add margin to the global obstacles to insure that every part of robot will not go into it. Here, we use binary dilation to increase the area of obstacles with the structure we defined. We defined the increasing margin is a 3x3 strcuting element with connectivity 2, which is:
 \begin{array}{cc} 
1 & 1 & 1\\
1 & 1 & 1\\
1 & 1 & 1\\
\end{array}
If the original matrix is: 
\begin{array}{cc} 
0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
\end{array}
After applying a margin of one on it, it will be:
\begin{array}{cc} 
0 & 0 & 0 & 0 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 0 & 0 & 0 & 0\\
\end{array}
After applying a margin of two on it, it will be:
\begin{array}{cc} 
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
\end{array}

The final margin added is the size of robot in pixel distance.

Below figures shows the effect of adding margin,
<table><tr>
<td> <img src="Report_Figures/map1.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 1</b></p></td>
<td><img src="Report_Figures/no_margin_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>No Margin</b></p></td>
<td><img src="Report_Figures/add_margin_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Add Margin</b></p></td>
</tr></table>
<table><tr>
<td> <img src="Report_Figures/map2.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 2</b></p></td>
<td><img src="Report_Figures/no_margin_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>No Margin</b></p></td>
<td><img src="Report_Figures/add_margin_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Add Margin</b></p></td>
</tr></table>
<p style='text-align: center;'><b>Figure 8: Addition of Margins to Obstacles</b></p>

In [ ]:
def compute_pixel_to_distance(self):
        # for get pixel_to_ditance value, use function "get_pixel_to_distance"
        # aruco marker:71mmx71mm
        # pixel_to_distance:return real world distance between two neighborhood pixels;unit:mm
        pixel_to_distance = 71 / (self.side_length*2)  # unit:mm
        
        self.pixel_to_distance = pixel_to_distance

        return self.pixel_to_distance

By knowing the real-world distance and pixel distance of aruco marker's side length, we can compute the conversion rate between pixel and real-world distance. 

#### 3.1.4 Make it more robust

In [ ]:
 def avg_globalmap(self,cap,avg_number=10):
        #average first avg_number image to construct global map instead of only use one image
        #in order to get rid of camera unstable problem
        #return self.globalmap
        weight = cap.get(3)
        height=cap.get(4)
        avg_map = np.zeros((int(weight), int(height)))
        for i in range(avg_number):
            ret, img = cap.read()  # returns ret and the frame
            globalmap = self.compute_globalmap(img)
            avg_map = np.add(globalmap, avg_map)
        avg_map = np.divide(avg_map, avg_number)
        avg_map[avg_map >= 0.5] = 1
        avg_map[avg_map < 0.5] = 0
        self.globalmap= avg_map
        return self.globalmap

When we tested our code, we found that only taking one frame from camera to construct the global map is unstable, because sometimes the frame taken is not in a good condition which result in generating a bad global map. Therefore, instead of only taking one frame, we take ten frames, and each frame computes one global map. After, we take the average of these ten global maps. Then, if the pixel is equal or larger than 0.5, which means that most of the frames consider this pixel as the obstacle, so we set the pixel to 1 (occupied). If the pixel is smaller than 0.5, which means that most of the frame consider this pixel as the free space, so we set the pixel to 0 (free).

### 3.2 Compute vertices

In [ ]:
def compute_vertex(globalmap):

    globalmap = np.uint8(np.dot(np.transpose(globalmap), 255))
    height,width=np.shape(globalmap)

    # apply canny edge detection
    edges = cv2.Canny(globalmap, 60, 160)

    # apply morphology close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # get contours
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # get vertices
    vertices = []
    for i in range(len(contours[0])):
        peri = cv2.arcLength(contours[0][i], True)
        approx = cv2.approxPolyDP(contours[0][i], 0.01 * peri, True)
        vertice = []
        s=10 #margin for boarder of map
        for i in range(len(approx)):
            #push away the vertex at the border of map to make sure robot will not go to border
            if approx[i][0][0]>(0+s):
                if approx[i][0][0]<(width-s):
                    if approx[i][0][1]>(0+s):
                        if approx[i][0][1]<(height-s):pass
                        else:approx[i][0][1]=approx[i][0][1]=10*height
                    else:approx[i][0][1]=approx[i][0][1]=-10*height
                else:approx[i][0][0]=approx[i][0][0]=10*width
            else:approx[i][0][0]=approx[i][0][0]=-10*width
            vertice.append(approx[i][0].tolist())
        vertices.append(np.array(vertice))

    return vertices

To compute the vertixes of shapes in the image, we first find edges of each shape by Canny edge detection [3]. It first calculate the edge gradient and direction for every pixel. The direction of gradient which perpendicular to the edge will have a large magnitude because the change of pixel intensity is large at the edge. Then, in order to thin the edge, it perform non-maximum suppresion. For each pixel, it checked wheather it is the local maximum of its neighbourhood in its gradient direction. If it is, then keep this pixel, otherwise set pixel to 0. Then, using Hysteresis Thresholding to decide which are edge. We set one maximum value and one minimum value. If the intensity gradient is larger than maximum value, it is considered as edge. If the intensity gradient is smaller than minimum value, it is discarded. If the value is between maximum and minimum value, we check weather it is connected to the edge which is larger than maximum value. If yes, it is considered as edge, otherwise, it is discarded.

Then, we finds countours of each shape. Since the contours are the boundary of shapes with same intensity, we can find countours by seraching each points [4]. If the point has pixel value 1 (1 is object), and its 4-neighbours or 8-neighbours contain pixel value 0 (0 is background), the point will be considered as the boundary. The algorithm will find both inner boundary and outer boundary. Here, we only use outer boundary. Also, to save memory, we only store the start and end point of each detected boundary, since all the boundaries of global obstacles are line. 

However, since the detected boundary is not so smooth, for example a rectangle, rather than only getting four corners, the ourput of countours also gave some unwanted points. Therefore, we use "approxPolyDP" to solve this problem. ApproxPolyDP function is an implementation of Douglas-Peucker algorithm [5] which can simplify the curve with fewer points. First, it considered the first point and last point are in the final curve. Then, it finds the farthest point from the line segment of first and last point. If the distance to farthest point is smaller than epsilon, the point is discarded. If the distance is larger then epsilon, the point are selected to the final curve. Then, recursively doing same process with start and the point selected, and then the last point and the point selected. Here, we set epsilon to the 0.01 of each contour's perimeter.

Finally, the vertices of shapes are the points selected to the final curve.

<table><tr>
<td> <img src="Report_Figures/map1.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 1</b></p></td>
<td><img src="Report_Figures/add_margin_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Add Margin</b></p></td>
<td><img src="Report_Figures/edges_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Canny Edge</b></p></td>
<td><img src="Report_Figures/contours_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Contours</b></p></td>
<td><img src="Report_Figures/vertices_map1.png" width="160" height="208;"/><p style='text-align: center;'><b>Vertices</b></p></td>
</tr></table>

<table><tr>
<td> <img src="Report_Figures/map2.png" style='text-align: left' width="160" height="208;"/><p style='text-align: center;'><b>Map 2</b></p></td>
<td><img src="Report_Figures/add_margin_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Add Margin</b></p></td>
<td><img src="Report_Figures/edges_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Canny Edge</b></p></td>
<td><img src="Report_Figures/contours_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Contours</b></p></td>
<td><img src="Report_Figures/vertices_map2.png" width="160" height="208;"/><p style='text-align: center;'><b>Vertices</b></p></td>
</tr></table>

<p style='text-align: center;'><b>Figure 9: Obstacle Treatment Step</b></p>



### 3.3 Detect Robot Position and Angle

In [1]:
def get_robot_position(image,robot_id=0):
    #Detect robot position by aruco marker
    #Return robot position; if not detected return None
    #robot_id: robot aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    robot_position = None
    angle=None

    #Detect Aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    if ids is not None:
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==robot_id:
                robot_position=centerpoint
                angle=angle_calculate(topRight,topLeft)
                break
            else:pass
    else:pass
    return robot_position,angle

We also use ArUco marker to detect the position of robot in real-time, since it's fast and robust.

In [ ]:
def angle_calculate(pt1,pt2):  
    #Get robot angle
    x=pt2[0]-pt1[0] 
    y=pt2[1]-pt1[1]
    angle=np.arctan2(y,x)
    return angle

Since each corner of marker is identified unequivocally, we can calculate the robot angle based on the angle of two corners.

## 4. Global Navigation

### 4.1 Compute Shortest Path

As stated in the introduction, the goal of the global navigation is to compute the shortest path from the start to the goal using the image acquired by the camera. To do so, a library (PyVisGraph) is used. PyVisGraph first builds a visibility graph from the vertices of the obstacles and, then, uses Dijsktra's algorithm to find the shortest path.

The coordinates of the start and goal as well as the coordinates of the vertices are given as output of the vision part. The vertices are formatted into an array which composition is as follows : Each row corresponds to a different obstacle and each element in the row corresponds to the coordinates (x & y) of the vertices of the obstacle.

Knowing the vertices and the obstacles they form, it is possible to build a visibility graph. The latter determines which vertices are visible from each vertex of each obstacle. Having the coordinates of each vertex, the Euclidean norm of the segment linking two vertices can be used to determine their distance to each other. The problem of collision with obstacles is avoided by adding margins on the obstacles dimensions in the vision part directly.

<img src="Report_Figures/Visibility_Graph.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 10: Visibility Graph</b></p>

<font color='red'>INCLUDE FIGURE VISIBILITY GRAPH</font>

As mentioned earlier, Dijkstra's algorithm is used to compute the shortest path. Besides its easy implementation, this method has several advantages: It guarantees to find the shortest path and is relatively efficient (and can, therefore, be applied to fairly large problems). However, the path computed is not optimal (the robot could, sometimes, have a more direct path than traveling from vertex to vertex).

The algorithm visits each vertex once beginning from the goal point to which it assigns a distance of zero. It computes the distance from the vertex it is currently visiting to each of the vertices preceding it (the vertices being linked to the current one in the visibility graph on the side of the starting point) and attributes them the distance computed. When the distance from the goal to a vertex is computed several times, only the shortest distance to it is attributed to this specific vertex. Once the distances are computed, the algorithm moves to the unvisited node with the lowest distance attached to it and repeats the process until reaching the starting point. 

Finally, the total distance from the starting point to the goal using the shortest path is the distance assigned to the starting point. As the algorithm keeps track of the shortest distance to each of the vertices at each stage, the shortest path can be found easily.

<img src="Report_Figures/Dijkstra_Pseudo_Code.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 11: Dijkstra's Pseudo Code</b></p>

### 4.2 Follow Shortest Path

Our global navigation control system uses a proportional controller to set the direction of the robot. Moreover, the robot speed is reduced to zero if the angle required for rotation is too big, else it is 100mm/s. 

<img 
src="Report_Figures/schema_speeds.png" style='text-align: center' width="550" height="208;"/><p style='text-align: center;'><b>Figure 12: Robot rotation schema</b></p>

The robot speed is the mean of the speed of the left motor and the right motor.
The angle is set by the difference of speed of the left/right motor. It isn’t modified by the offset speed as you can see in these equations:

$Speed_{left}=\delta angle \cdot Radius \\
Speed_{right}=\delta angle \cdot (Radius+Diameter) \\
\delta angle= \frac{Speed_{right}-Speed_{left}}{Diameter}\\
Mean_{speed}=\delta angle \cdot (Radius+\frac{Diameter}{2})=\frac{Speed_{left}+Speed_{right}}{2}$

It permits us to control both of the direction and speed of the robot separately.

To follow the path, we just have a global variable which set the position in the path (list of points) so that we have a point goal. Every iteration we study the distances between points on the paths to eventually change goal. If we end the list, we arrived.

We add a coefficient of 1.5 to the motors target to have the right speed. We saw that this coefficient is dependent of the speed, but we approximate to 1.5 for simplicity. Also, we bounded the speeds to +-400mm/s. 
Then the speeds are transmit to the navigation() function, which control the motors.

### 4.3 Choice of Controller

We didn’t add some integration (PI) because we founded the system smooth already. We added some of the simulations that we made to design the controller. Even if there is not noise and error in displacement it gives an idea of time of reaction and smoothness of the motors when we change path point. We see that if we want to minimize the error (distance between the robot and the path) we have to set a strong controller which correct fast. This implies a big variation of the speeds of the motors, the inputs of the system. With a smooth correction we don’t achieve to avoid large pics as you can see in these pictures. 

We also tried to adapt the speed to the angle and derivate speed, as you can see in the second example.

One can note that here we simulated as 1 second long loop. It is a margin for the time taken to obtain a picture by the camera. Now we know that this time is very over evaluated but the simulations are still pertinent.

<table><tr>
<td><img src="Report_Figures/map_PID_50.png" width="160" height="208;"/><p style='text-align: center;'><b>map</b></p></td>
<td><img src="Report_Figures/speed_PID_50.png" width="160" height="208;"/><p style='text-align: center;'><b>speeds</b></p></td>
</tr></table>
<p style='text-align: center;'><b>50mm/s speed, PID controller (still improvable) evaluated 1 time/second</b></p>

<table><tr>
<td><img src="Report_Figures/map_PID_100.png" width="160" height="208;"/><p style='text-align: center;'><b>map</b></p></td>
<td><img src="Report_Figures/speed_PID_100.png" width="160" height="208;"/><p style='text-align: center;'><b>speeds</b></p></td>
</tr></table>
<p style='text-align: center;'><b>100mm/s speed, PID and speed adaptation evaluated 1 time/second</b></p>

<table><tr>
<td><img src="Report_Figures/map_P_100.png" width="160" height="208;"/><p style='text-align: center;'><b>map</b></p></td>
<td><img src="Report_Figures/speed_P_100.png" width="160" height="208;"/><p style='text-align: center;'><b>speeds</b></p></td>
</tr></table>
<p style='text-align: center;'><b>100mm/s speed, P controller evaluated 1 time/second</b></p>

100mm/s speed P evaluated 1 time/second
We see that obviously the P is less accurate but the pic in the PID show that we can’t avoid some hard reactions if we want to rest in the path. 
To conclude we just made a Proportional controller because without speaking about the computational inclusion of derivative and integrative calculation it wasn't optimal to precisely follow the path in our case. By observation we also are satisfied about the fluidity of the robot displacements.


In [ ]:
def global_nav(path):
    global state,index,direction,speed_left,speed_right
    
    #constants
    v_max=400#mm/s
    Diameter=94#mm
    goal_margin=20#mm
    threshold=0.3#rad
    Kp=0.8
    
    #CHECK GOAL
    if math.sqrt((robot_state[0]-path[index][0])**2+(robot_state[1]-path[index][1])**2)<goal_margin:
        if index+1>=int(np.size(path)/2):
            print('arrived')
            state=2
            return 0,0
        else:
            index=index+1
            print('index : ',index)
    
    #MOTORS
    #calculate angle correction
    obj_angle=np.arctan2(path[index][1]-robot_state[1],path[index][0]-robot_state[0])
    delta_angle=obj_angle-direction
    while(delta_angle>np.pi):
        delta_angle=obj_angle-np.pi
    while(delta_angle<=-np.pi):
        delta_angle=obj_angle+np.pi
        
    #mean speed
    if abs(delta_angle)>threshold:
        vitesse=0
    else:
        vitesse=100
    
    #speed_left/right according to angle correction
    speed_left=(Kp*(delta_angle))*Diameter
    speed_right=-(Kp*(delta_angle))*Diameter
    
    #bounded speed for turn
    if speed_right<-v_max:speed_right=-v_max
    if speed_right>v_max:speed_right=v_max
    if speed_left<-v_max:speed_left=-v_max
    if speed_left>v_max:speed_left=v_max
    
    #bounded speed for turn+speedDC
    delta_speed=abs(speed_left-speed_right)
    if speed_left+vitesse>v_max:#PB si les deux au dessus
        speed_left=v_max
        speed_right=v_max-delta_speed
    elif speed_right+vitesse>v_max:
        speed_left=v_max-delta_speed
        speed_right=v_max
    else:
        speed_left=speed_left+vitesse
        speed_right=speed_right+vitesse
    
    return speed_left,speed_right

## 5. Filtering

The Kalman function is dedicated to fusing and filtering the output of two sensors (the camera and the sensors on the motors) in order to improve the precision of the localization of the thymio. The Kalman filter is implemented using the correponsding functions from the filterpy library. To do so, we determine 5 matrixes that describe our system:

-	The matrix F describes the relation between the current state and the next state. 
-	The matrix H describes the relation between the state and the measurements
-	The matrix P is a state covariance matrix. It is a diagonal matrix which elements are the variance of each state.
-	The matrix P gives information about the noise of each measurement. It is also a diagonal matrix, which elements are the variance of each measured value.
-	The matrix Q is related to the process noise (which we decided to set as discrete white noise).

The next step consists in using the different sensors to obtain measurements.

Two functions are used to do a prediction about a state, to update the measurements and to filter the state. The function output is the filtered state.

Our state robot_state has four components: x and y position and x and y speed ($v_x$ and $v_y$). Our measurements also have four components: x and y position measurements from the camera that is updated in each iteration of the main loop (as a new image is being taken) as well as $v_x$ and $v_y$ measured directly on the motors. 

The motor speeds are given by left and right speed. We use the direction of the robot which is a global variable, to transform them into x and y speeds. 

### 5.1 Hidden camera scenario

We wanted our Kalman to still update the state of the robot correctly even when the camera is temporarily hidden. To do so, we add a condition to see if a new image has been taken. If no new image is available, the Kalman doesn’t use the position measurement from the camera which is outdated and would false the filtering. Instead, we use the current x and y position of the robot from robot_state instead of the position measurement as a temporary measure. Once the camera is back on, the position measurements from the camera will be used again. 

### 5.2 Kalman parameters 

The variances of our state position and speed were tuned by trial and error during the testing of the Kalman filter with the rest of the code.

The variance of the measurements, both from the camera and the motors were determined experimentally. We implemented a python code that records 1000 measurements from either the camera or the motor sensors and calculates the variance. Those measurements were made while the Thymio kept either the same position or the same speed, depending on the value measured by the sensor. The variance found varied each time we run our code. So we dediced to take the mean of the several resulting variances.

<font color='red'>REMARK : BETTER EXPLAIN 1000 MEASUREMENTS</font>


## 6. Motor Control

## 7. Local Navigation

Navigation is the function that controls the state of the robot and applies the corresponding speed on the two motors. It is called once in each iteration of the main loop. There are two states: state 0 and state 1.

In state 0, the robot is in global navigation mode and follows the global_path trajectory given as input to the navigation function. When in state 0, the global_nav() function is called with the path as input. The speeds given by global_nav() are then applied to the motors.

<font color='red'>CAREFUL ABOUT SEPARATION BETWEEN MOTOR CONTROL AND LOCAL NAVIGATION</font>

In state 1, the robot is avoiding a 3D obstacle that was added and, therefore, not recorded on the global map. Each time navigation() is called, the prox.horizontal values of the thymio are read. We compute the normalized weighted sum of the two sensor values on the left (prox[0] and prox[1]) on the one hand and of the two right sensor values (prox[3] and prox[4]). The sum is weighted (weight of 1/2 for the proximity sensor on the far right/center right) to give more importance on the measurements given by sensors closer to the central axis of the robot. Indeed, an obstacle being closer to this axis will require a higher deviation from the initial path to avoid it.

The velocity of the left (right) motor is computed by subtracting the result of the weighted sum of measurements given by the left (right respectively) proximity sensors multiplied by a coefficient and subtracted to a constant value. This way, the left prox sum is contributing to the left motor speed and the right prox sum to the right one. In turn, the robot turns in the opposite direction of the detected obstacle.

The switching between the different states is done with the measured values of the proximity sensors. If at least one of the four exterior sensor values is above a certain threshold, the robot switches to state 1. If all the proximity values are below another set threshold, the state switches back to 0. 

We experienced problems when merging the global and local navigation. Depending on the geometry and the placement of the 3d obstacle in relation to the global path, the robot would sometimes get stuck. As a matter of fact, it was trying to avoid the obstacle but, as soon as the prox value was below the threshold, it would turn back towards the obstacle when trying to follow the global path. This meant the thymio was constantly changing state and switching from two opposite objectives: turning one direction to avoid the obstacle and turning the opposite direction to reach its goal. 

In order to tackle this error, when the robot switches back to state 0 from state 1, meaning when the obstacle is no longer detected, the global path towards the goal is recomputed with a starting point in front of the thymio. As a result, when the robot turned enough to avoid the obstacle, it goes straight to a new starting point and follows its new trajectory towards the goal, this time from a place far enough from the obstacle. In turn, the path given as input to the global_nav() function when the robot is in state 0 is either the original shortest path calculated during the initialization outside the main loop, or the new path calculated when moving away from a 3d obstacle. 

<font color='red'>INCLUDE FIGURE LOCAL NAVIGATION</font>

## 8. Code

## 9. References

[1] OpenCV. <i>Image Thresholding</i>. https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

[2] OpenCV. <i>Detection of ArUco Markers</i>. https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html

[3] OpenCV. <i>Canny Edge Dectecion</i>. https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html

[4] OpenCV. <i>Contours : Getting Started</i>. https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html

[5] https://en.wikipedia.org/wiki/Ramer%E2%80%93Douglas%E2%80%93Peucker_algorithm

[6] https://github.com/TaipanRex/pyvisgraph

[7] Basics of Mobile Robotics

[8] http://cs.indstate.edu/~rjaliparthive/dijkstras.pdf

[9] https://courses.lumenlearning.com/waymakermath4libarts/chapter/shortest-path/